## Intro

In [1]:
%load_ext autoreload 

import numpy as np
import pandas as pd
# from aocd i
# import get_data
import os
import re
import time
import networkx as nx
from heapq import *
import functools
from itertools import product
from  math import log
from tqdm import tqdm
import sys


module_path=os.path.abspath(os.path.join('../'))
sys.path.append(module_path)
from ComplexMap import ComplexMap


In [2]:
pad1_str = "789;456;123;B0A"
pad1 = ComplexMap(pad1_str.split(";"),str)
_ = pad1.print_me()

pad1_coords_rev = pad1.map_as_dict()
pad1_coords = {v:k for k,v in pad1_coords_rev.items()}


789
456
123
B0A


In [3]:
pad2_str = "B^A;<v>"
pad2 = ComplexMap(pad2_str.split(";"),str)
_ = pad2.print_me()

pad2_coords_rev = pad2.map_as_dict()
pad2_coords = {v:k for k,v in pad2_coords_rev.items()}


B^A
<v>


In [4]:
mapping = {">":1j,"<":-1j,"v":1,"^":-1}
mapping_rev={v:k for k,v in mapping.items()}

def get_directions_str(directions_as_complex):
    return "".join([mapping_rev[d] for d in directions_as_complex])

assert get_directions_str([-1,-1,1j,1]) == "^^>v"

In [5]:
@functools.cache
def good_sequence(seq,avoid):
    return avoid not in { sum(seq[:i]) for i,_ in enumerate(seq)}

In [6]:
@functools.cache
def expand_num_set(num,avoid):
    i_res = [int(np.sign(num.imag))*1j]*int(abs(num.imag))
    r_res = [int(np.sign(num.real))*1]*int(abs(num.real))

    res = set([tuple(i_res + r_res), tuple(r_res + i_res)])
    res = {seq for seq in res if good_sequence(seq,avoid)}
            
    return res 

# for t in [-2+1j,-1,1,1j,2j,4j+1,-15+3j]:
#     print(f"{t} :: {expand_num_set(t,1)}")

In [7]:
def get_paths_nonnested(pad,pad_coords,pad_coords_rev):
    G_pad = nx.from_edgelist(pad.get_edges_for_graph(vals_to_consider="B",includes_given=False))
    
    paths_pad_values = {}
    paths_pad_coords = {}
    
    pos_b=pad_coords["B"]
    for ns, ne in product(G_pad.nodes, repeat=2):
        if ns == ne:
            directions = {"A"}
        else:
            d = ne-ns
            directions = expand_num_set(d,pos_b-ns)
            directions = {get_directions_str(d)+"A" for d in directions}
            
        # paths_pad_coords[(ns,ne)]=directions
        paths_pad_values[(pad_coords_rev[ns],pad_coords_rev[ne])]=directions
    
    return paths_pad_values
    

In [8]:
paths_pad1_values = get_paths_nonnested(pad1,pad1_coords,pad1_coords_rev)
len(paths_pad1_values)

121

In [9]:
paths_pad2_values = get_paths_nonnested(pad2,pad2_coords,pad2_coords_rev)
len(paths_pad2_values)

25

# V1 of Part A (suboptimal)

In [10]:
def leave_shortest(in_set):
    k=min({len(x) for x in in_set})
    return {x for x in in_set if len(x) == k}

# @functools.cache
def expand_level_str(in_str,dict_for_expansion):
    in_str_prefixed = "A"+in_str
    res = [""]
    for prev_char, this_char in zip(in_str_prefixed[:-1],in_str_prefixed[1:]):
        res = set([pref+suff for pref in res for suff in dict_for_expansion[(prev_char,this_char)]])
    return leave_shortest(res)

def expand_level_set(in_set,dict_for_expansion):
    res_set = set()
    for in_str in in_set:
        res_tmp = expand_level_str(in_str,dict_for_expansion)
        res_set.update(res_tmp)
    return leave_shortest(res_set)

In [11]:
def add_nested_level_paths(paths_prev_level,dict_for_expansion):
    this_level_paths = {}
    for (ns,ne),val in paths_prev_level.items():
        val2 = expand_level_set(val,dict_for_expansion)
        this_level_paths[(ns,ne)] = val2
    return this_level_paths

In [12]:
levels = [paths_pad1_values]
for i in range(2):
    next_level_paths = add_nested_level_paths(levels[i],paths_pad2_values)
    levels.append(next_level_paths)


In [13]:
def get_str(in_str,dict_for_command):
    res_set = expand_level_str(in_str,dict_for_command)
    return res_set.pop()

In [14]:
def partA(file_name):
    with open(file_name) as f:
        lines = list(map(str.strip,f.readlines()))
    
    res = [len(get_str(x,levels[2]))*int(x[:-1]) for x in lines]
    
    return sum(res)

In [15]:
%time assert partA("test.txt") == 126384 #wrong because it allows arm to go over Blank
print("All ok")

CPU times: total: 0 ns
Wall time: 8 ms
All ok


In [16]:
%time assert partA("input.txt") == 211930
print("All ok")
# 213458 is too high

CPU times: total: 15.6 ms
Wall time: 11 ms
All ok


# Part B and v2 of Part A (more optimal)

In [17]:
@functools.cache
def getShortestLength(pattern,level,max_level,max_length=np.Inf):
    if level == max_level:
        return len(pattern)
    else:
        pattern_prefixed = "A"+pattern
        path_values = (paths_pad2_values if level > 0 else paths_pad1_values)
        res = 0
        for prev_char, this_char in zip(pattern_prefixed[:-1],pattern_prefixed[1:]):
            possible_expansions = path_values[(prev_char,this_char)]
            expansion_length = np.Inf
            for expansion in possible_expansions:
                tmp = getShortestLength(expansion,level+1,max_level,max_length=expansion_length)
                expansion_length = min(tmp,expansion_length)
            res+=expansion_length
            
            if res > max_length:
                # abort
                return np.Inf
        return res

In [18]:
assert getShortestLength("029A",0,3) == 68 == len(get_str("029A",levels[2]))
assert getShortestLength("029A",0,1) == 12 == len(get_str("029A",levels[0]))

In [19]:
def partA_alternative(file_name):
    with open(file_name) as f: 
        lines = list(map(str.strip,f.readlines()))

    res = [getShortestLength(x,0,3)*int(x[:-1]) for x in lines]

    return sum(res)

In [20]:
def partB(file_name):
    with open(file_name) as f:
        lines = list(map(str.strip,f.readlines()))
    
    res = [getShortestLength(x,0,26)*int(x[:-1]) for x in lines]
    
    return sum(res)

In [21]:
%time assert partA_alternative("test.txt") == 126384 #wrong because it allows arm to go over Blank
print("All ok")

CPU times: total: 0 ns
Wall time: 2 ms
All ok


In [22]:
%time assert partA_alternative("input.txt") == 211930
print("All ok")
# 213458 is too high

CPU times: total: 15.6 ms
Wall time: 1 ms
All ok


In [23]:
%time assert partB("input.txt") == 263492840501566
print("Part B OK")

CPU times: total: 0 ns
Wall time: 11 ms
Part B OK
